# 🚀 Proyecto ELT con SpaceX API  
## Notebook 01 – Extracción y Carga

En este notebook se realiza la **extracción de datos desde la API pública de SpaceX** y el **almacenamiento en Delta Lake**.  

Se siguen los pasos de la consigna:  
1. Extracción de **2 o más endpoints**.  
2. Uso de al menos un **endpoint dinámico (actualizable)** y otro **estático**.  
3. Guardado en **formato Delta Lake**.  
4. Aplicación de **extracción incremental y full** según corresponda.  


In [1]:
# =========================
# CELDA DE CONFIGURACIÓN INICIAL
# =========================

import sys
from pathlib import Path

# --- RUTA DEL PROYECTO ---
# Detecta automáticamente la raíz del proyecto buscando la carpeta "src"
def find_project_root(marker="src"):
    path = Path().cwd()
    for _ in range(5):  # sube hasta 5 niveles si es necesario
        if (path / marker).exists():
            return path
        path = path.parent
    raise FileNotFoundError(f"No se encontró la carpeta '{marker}' en los niveles superiores.")

project_root = find_project_root()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"Project root detectado en: {project_root}")

# --- IMPORTS DEL PROYECTO ---
from src.extract import fetch_data
from src.load import save_to_parquet
from src.config import setup_logger
logger = setup_logger()

Project root detectado en: c:\Users\MONSO\OneDrive\Escritorio\Final-DataEngineering


## 🔗 Endpoints seleccionados

- **Dinámicos:**  
  - `launches/upcoming` → Próximos lanzamientos.  

- **Estáticos:**  
  - `rockets` → Información de cohetes.  
  - `dragons` → Información de cápsulas Dragon.  

In [2]:
# Extracción FULL de endpoints estáticos
logger.info("Extrayendo Rockets (FULL)...")
rockets_df = fetch_data("rockets")
print(f"✅ Extracción completa: {len(rockets_df)} rockets extraídos.")

logger.info("Extrayendo Dragons (FULL)...")
dragons_df = fetch_data("dragons")
print(f"✅ Extracción completa: {len(dragons_df)} dragons extraídos.")

2025-09-06 21:30:08,815 - INFO - Extrayendo Rockets (FULL)...
2025-09-06 21:30:12,208 - INFO - Extrayendo Dragons (FULL)...


✅ Extracción completa: 4 rockets extraídos.
✅ Extracción completa: 2 dragons extraídos.


In [3]:
# Extracción INCREMENTAL de upcoming launches
logger.info("Extrayendo Upcoming Launches (INCREMENTAL)...")

# Extracción de datos nuevos
launches_new = fetch_data("upcoming_launches")

print(f"✅ Extracción completa: {len(launches_new)} nuevos lanzamientos extraídos.")

2025-09-06 21:30:13,616 - INFO - Extrayendo Upcoming Launches (INCREMENTAL)...


✅ Extracción completa: 18 nuevos lanzamientos extraídos.


## 💾 Guardado

Se guarda cada DataFrame en formato **Delta Lake**:  
- Los **endpoints dinámicos** (`upcoming_launches`) se almacenan con **particiones por fecha (extracción incremental)**.  
- Los **endpoints estáticos** (`rockets`, `dragons`) se guardan en una única ruta (extracción full).  

In [ ]:
# Guardado de datos en Parquet

# Carga de datos con un "modo full" en formato Parquet
save_to_parquet(rockets_df, "rockets", layer="bronze")
save_to_parquet(dragons_df, "dragons", layer="bronze")

# INCREMENTAL save de Launches
if not launches_new.empty:
    print(f"✅ Extracción completa: {len(launches_new)} launches nuevos extraídos.")
    # Guardar solo los datos incrementales
    save_to_parquet(launches_new, "upcoming_launches", layer="bronze", incremental=True)
else:
    logger.info("No hay datos nuevos para upcoming_launches")

logger.info("Todos los datasets fueron guardados correctamente en Parquet.")

TypeError: get_partition_path() takes 2 positional arguments but 3 were given